# Course: Deep Learning
# Author: Sandro Camargo <sandrocamargo@unipampa.edu.br>
# Classification with Multi Layer Perceptron Example Urban Land Cover
# Dataset: https://archive.ics.uci.edu/ml/datasets/Urban+Land+Cover

A Python library is a collection of related functions. A library contains bundles of encapsulated code which can be used repeatedly in different programs.

In [ ]:
# Import Libraries
import keras # Neural Network Library
from keras import layers # Layers to a neural network
from keras import optimizers # optimizers
import pandas as pd # Data Manipulation library
import numpy as np # Fast Numeric Computing library
import tensorflow as tf # Optimizers
import matplotlib.pyplot as plt # Plot library
from sklearn.preprocessing import MinMaxScaler, label_binarize
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, ConfusionMatrixDisplay
from tensorflow.keras.utils import plot_model # Print the network

In [ ]:
import urllib.request
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00295/Urban%20land%20cover.zip", "urbanlandcover.zip")

In [ ]:
import zipfile
with zipfile.ZipFile("urbanlandcover.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:
# Loading training dataset
data = pd.read_csv('training.csv', delimiter=",", header=0)
# About the parameters
# Header=1: column names (day, month, year, ...) are in the line 1 of this CSV file.
# skiprows=[124,125,126,170]: this lines, which not contains valid data, are not imported. If this parameter is missing, all lines are imported.
# usecols=list(range(0,13)): The last column, which is named Classes, is not imported. If this parameter is missing, all columns are imported.

# inspecting columns and data types from "data" dataframe
data.info()

In [ ]:
datatest = pd.read_csv('testing.csv', delimiter=",", header=0)
datatest.info()

In [ ]:
classes = data.iloc[:,0]
data.drop(data.columns[[0]], axis=1, inplace=True)
scaler = MinMaxScaler()
print(scaler.fit(data))
MinMaxScaler()
data = pd.DataFrame(scaler.transform(data))

In [ ]:
classestest = datatest.iloc[:,0]
datatest.drop(datatest.columns[[0]], axis=1, inplace=True)
scalertest = MinMaxScaler()
print(scalertest.fit(datatest))
MinMaxScaler()
datatest = pd.DataFrame(scaler.transform(datatest))

The dataset must be randomly splitted in two parts: training set and testing set. The main approaches to split are holdout and n-fold cross validation.
*   Training set is used for building (training) the model.
*   Testing set is used for testing the generalization ability of the model built.

Moreover, inputs($x$) and outputs($y$) must be splitted in each set.




In [ ]:
# Creating the training dataset
train_x = data
train_y = label_binarize(classes, classes=['tree ', 'grass ', 'soil ', 'concrete ', 'asphalt ', 'building ', 'car ', 'pool ', 'shadow '])

# Creating the testing dataset
test_x = datatest
test_y = label_binarize(classestest, classes=['tree ', 'grass ', 'soil ', 'concrete ', 'asphalt ', 'building ', 'car ', 'pool ', 'shadow '])

# Verifying dataset dimensions
print('The training dataset (inputs) dimensions are: ', train_x.shape)
print('The training dataset (outputs) dimensions are: ', train_y.shape)
print('The testing dataset (inputs) dimensions are: ', test_x.shape)
print('The testing dataset (outputs) dimensions are: ', test_y.shape)

After creating the datasets, the next step is defining the architecture of our model.

It must be defined:


*   Architecture: in terms of neurons and layers
*   Optimizer: is the algorithm or method used to change the weights in order to minimize the loss function.

The last step is compiling the model. In this step the loss function, the optimizer and the evaluation metrics must be defined.

In [ ]:
# Function to define model architecture
def build_model():
  # Defining the architecture
  # Sequential = Feedforward Neural Network
  # 1 single neuron
  # input_shape is the amount of columns from training set
  model = keras.Sequential([
        layers.Input(shape=[len(train_x.columns)]),
        layers.Dense(9, activation="relu"),
        layers.Dense(9, activation="softmax")
  ])

  # Defining the optimizer
  optimizer = tf.keras.optimizers.RMSprop(
      learning_rate = 0.001)

  # Mean Squared Error (MSE) is the default loss function in regression models
  model.compile(loss = 'categorical_crossentropy',
      optimizer = optimizer,
      metrics = ['categorical_crossentropy','accuracy'])

  return model

Just for curiosity, you should observe how many parameters ($\theta$) your model has.
At this point, your model is built.

In [ ]:
model = build_model()
model.summary()

After creating the model, it must be trained (fitted).
Training is done using training set and the amount of epochs must be defined.

In [ ]:
EPOCHS = 50

history = model.fit(
    train_x, train_y, epochs = EPOCHS, verbose = 1
)

This plot should be generated just to inspect the learning convergence.
It is expected a decreasing of the loss function value through the epochs.


In [ ]:
plt.plot(history.history['categorical_crossentropy'])
plt.title('Training Categorical Cross Entropy')
plt.ylabel('Categorical Cross Entropy')
plt.xlabel('Epoch')
plt.legend(['Error'], loc='upper right')
plt.savefig("mlp-class-lossfunction.png")
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Training Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy'], loc='lower right')
plt.savefig("mlp-class-trainingaccuracy.png")
plt.show()

In [ ]:
weights = model.get_weights() # return a numpy list of weights
print(type(weights))
print(weights)

After the training process, the knowledge learnt by a neural network is stored in its weights.

After the training process, the model should be tested in order to measure its quality, it means, how good are its predictions. The model must be evaluated using the testing set, which is composed by samples that are not in the training set. In regression problems, the correlation coefficient is the default metric to measure the model quality.
The correlation coefficient is computed using real outputs ($y$) and predicted outputs ($\hat{y}$). Correlation coefficient can vary between 0 (bad predictions) and 1 (perfect predictions).

In [ ]:
test_predictions = model.predict(test_x) # predict randon activities with the built linear regression model
print(confusion_matrix(test_predictions.argmax(axis=1), test_y.argmax(axis=1)))
print(classification_report(test_predictions.argmax(axis=1), test_y.argmax(axis=1), target_names=['tree ', 'grass ', 'soil ', 'concrete ', 'asphalt ', 'building ', 'car ', 'pool ', 'shadow ']))
print('The accuracy on the test set is equal to: %.4f ' % accuracy_score(test_predictions.argmax(axis=1), test_y.argmax(axis=1)))

In [ ]:
cm = confusion_matrix(test_predictions.argmax(axis=1), test_y.argmax(axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['tree', 'grass', 'soil', 'conc', 'asph', 'build', 'car', 'pool', 'shad'])
disp.plot(cmap="bwr")
plt.title('Confusion Matrix for Testing Set')
plt.savefig("mlp-class-confusionmatrix.png")
plt.show()

In [ ]:
train_predictions = model.predict(train_x)
print(confusion_matrix(train_predictions.argmax(axis=1), train_y.argmax(axis=1)))
print(classification_report(train_predictions.argmax(axis=1), train_y.argmax(axis=1), target_names=['tree ', 'grass ', 'soil ', 'concrete ', 'asphalt ', 'building ', 'car ', 'pool ', 'shadow ']))
print('The accuracy on the training set is equal to: %.4f ' % accuracy_score(train_predictions.argmax(axis=1), train_y.argmax(axis=1)))

In [ ]:
# Save the model architecture as an image
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)